# Sandbox

## Config

In [1]:
# Import dependencies
import tekore as tk
import spotipy
import os
import numpy as np
import pandas as pd
import time
from datetime import datetime
from dateutil.relativedelta import relativedelta
# Environment variables
CLIENTID = os.environ.get('CLIENTID')
CLIENTSECRET = os.environ.get('CLIENTSECRET')
REDIRECT_URI = os.environ.get('REDIRECT_URI')

# Set global variables
MINDATE = datetime.today() - relativedelta(years=5)


## Authentication

In [2]:
### Tekore
# Get client token
app_token = tk.request_client_token(CLIENTID, CLIENTSECRET)
# Create spotify instance
spotify = tk.Spotify(app_token)

## Tekore Sandbox

In [3]:
# First let's try to get an artist's Spotify ID by searching for their name
# Searching by artist name returns a tuple with many different Spotify elements
artists = spotify.search('Sylvan Esso', types=['artist'], limit=10)
artists

(FullArtistOffsetPaging with fields:
   href = 'https://api.spotify.com/v1/search?offset=0&limit=10&query=Syl...'
   items = [10 x FullArtist(external_urls, followers, genres, href, ...)]
   limit = 10
   next = 'https://api.spotify.com/v1/search?offset=10&limit=10&query=Sy...'
   offset = 0
   previous = None
   total = 800,)

In [4]:
# The code above doesn't help get the artist's id because it returns a tuple and we cannot select the tuple's items like we would a list
artists.items[0]

AttributeError: 'tuple' object has no attribute 'items'

In [5]:
# The code below, though, does allow us to unpack the tuple
# Note the comma trailing the variable
# This tells python we know we are dealing with a tuple, and we can select the first artist from it
artists, = spotify.search("Sylvan Esso", types=['artist'], limit=10)
artists.items[0]

FullArtist with fields:
  external_urls = {'spotify'}
  followers = Followers(href, total)
  genres = [0 x str]
  href = 'https://api.spotify.com/v1/artists/39vA9YljbnOApXKniLWBZv'
  id = '39vA9YljbnOApXKniLWBZv'
  images = [3 x Image(height, url, width)]
  name = 'Sylvan Esso'
  popularity = 58
  type = 'artist'
  uri = 'spotify:artist:39vA9YljbnOApXKniLWBZv'

In [9]:
# From within the artist item we can select other elements like genre or artist_id
print("Sylvan Esso is classified under the following genres: " + ", ".join([x for x in artists.items[0].genres]))
print(f"Sylvan Esso's Spotify ID is: {artists.items[0].id}")

Sylvan Esso is classified under the following genres: 
Sylvan Esso's Spotify ID is: 39vA9YljbnOApXKniLWBZv


In [7]:
# Let's see what the artist API can tell us
# We can see the artist method again returns the FullArtist
sylvanEsso = spotify.artist('39vA9YljbnOApXKniLWBZv')
sylvanEsso
# Interesting that we can get popularity on a scale of 0-100; this will be useful later

FullArtist with fields:
  external_urls = {'spotify'}
  followers = Followers(href, total)
  genres = [0 x str]
  href = 'https://api.spotify.com/v1/artists/39vA9YljbnOApXKniLWBZv'
  id = '39vA9YljbnOApXKniLWBZv'
  images = [3 x Image(height, url, width)]
  name = 'Sylvan Esso'
  popularity = 58
  type = 'artist'
  uri = 'spotify:artist:39vA9YljbnOApXKniLWBZv'

In [10]:
# Let's see if we can get a list of ids for the artist's albums
sylvanAlbums = spotify.artist_albums('39vA9YljbnOApXKniLWBZv', limit=50)
print(len(sylvanAlbums.items))

50


In [202]:
# We have a problem: Spotify limits the number of albums to 50. 
# We will have to write some logic to get all the albums if there are more than 50
# The offset parameter will help
sylvanAlbums = spotify.artist_albums('39vA9YljbnOApXKniLWBZv', limit=50, offset=50)
print(len(sylvanAlbums.items))
# We can see there are 87 albums (50+37) as of this writing
# We can use a list iteration to get all the album ids
[x.id for x in sylvanAlbums.items][:5]

37


['3EOSrgN0wOOqNvS5GUsiLp',
 '6w0oK4Y6MvRqFaE3J0Uwyz',
 '0oTOOEPtYGZFYnWlLnOsxL',
 '3BBCxnIvQauRM4FT2Mc3pX',
 '1Aus1dFkP2BSyzibCl0HA7']

In [163]:
# Let's make a function to join the multiple pages of albums together
def get_artistAlbums(artistID):
    # Initialize variables
    length = 50
    albumList = []
    offset=0
    # If the returned page is 50 items that means we need to call the API for another chunk
    while length==50:
        # Call the API to get the artist's albums
        albums = spotify.artist_albums(artistID, limit=50, offset=offset)
        # Append this chunk to our existing list
        albumList = albumList + [x.id for x in albums.items]
        # Reset the length. If the length goes below 50 we will not call the API again
        length = len(albums.items)
        # Increase the offset so we are not getting the same albums again
        offset+=50
    return albumList

# It's working! Now we have all 87 albumIDs
sylvanAlbumList = get_artistAlbums('39vA9YljbnOApXKniLWBZv')
print(len(sylvanAlbumList))
sylvanAlbumList[:5]


87


['6rl4rzZxECegl9PdYG0hfk',
 '2KAPJ40On8JY1Yt64JJgTk',
 '68ClkAMsCDyx2lQMsnbY55',
 '1BJMCEXQ7JmuVlJ6cYbe3x',
 '1862qnxqsLeiNDblknjtiL']

In [159]:
# One of the requirements of a summer playlist is that all songs must be newer than three years old 
# We can include this logic in our list comprehension
[x.id for x in sylvanAlbums.items if datetime.strptime(x.release_date, '%Y-%m-%d')>MINDATE]

['3EOSrgN0wOOqNvS5GUsiLp',
 '6w0oK4Y6MvRqFaE3J0Uwyz',
 '0oTOOEPtYGZFYnWlLnOsxL',
 '3BBCxnIvQauRM4FT2Mc3pX',
 '1Aus1dFkP2BSyzibCl0HA7',
 '0duHxSsZOXo0rlxY2ZabO0',
 '3IBo4lqI2xcSEkEMl73Aop',
 '0LX2FkR9ARLv5PmVQ211jZ',
 '61PjEcLjGh1UvfsCkn5eB8',
 '5qrhRBImmdLDEm2W8Y4lE3',
 '3hJ8Qe8dTlKJwAt1VPNc7g',
 '6tfc4x6M0rRoizzWXggXGa',
 '5N5q1BjX8YE0hjhBnwJRMq',
 '00wKAsbWtTgG5uCq524Tvk',
 '6fM5eqkTrd7ReiwRPnRMzg',
 '1UKlb2Lb7CW1trU8ultvGr']

In [208]:
# We can also specify in the API call that we only want actual albums or singles with the "include_groups=['album']" argument
# Let's try this again. Make a function to join the multiple pages of albums together
def get_artistAlbums(artistID):
    # Initialize variables
    length = 50
    albumList = []
    offset=0
    # If the returned page is 50 items that means we need to call the API for another chunk
    while length==50:
        # Call the API to get the artist's albums
        albums = spotify.artist_albums(artistID, include_groups=['album', 'single'], limit=50, offset=offset)
        # Append this chunk to our existing list
        albumList = albumList + [x.id for x in albums.items if datetime.strptime(x.release_date, '%Y-%m-%d')>MINDATE]
        # Reset the length. If the length goes below 50 we will not call the API again
        length = len(albums.items)
        # Increase the offset so we are not getting the same albums again
        offset+=50
    return albumList

# Now we only have 15 albumIDs that are in the dates that we want and the type that we want
sylvanAlbumList = get_artistAlbums('39vA9YljbnOApXKniLWBZv')
print(len(sylvanAlbumList))
sylvanAlbumList


15


['6rl4rzZxECegl9PdYG0hfk',
 '2KAPJ40On8JY1Yt64JJgTk',
 '68ClkAMsCDyx2lQMsnbY55',
 '1iKIf39XkjsKa5xCKJmcN4',
 '3RP1mc5yIIwGCrdST276Lg',
 '1203zCqGjsMNe9vre3dMrJ',
 '1Ho3UkeW5DfE812uhE6Q9Q',
 '1UiIz58AGc20MDp1LMYsdu',
 '7m9kZfDONAl0UMYeQRFTNI',
 '0mZc9rzgNi9Jl0PbuU0G3H',
 '1n2SGGnNyAy6npmMNxIjuV',
 '4edAJKbnB3xCVsvXmhKvv0',
 '3PvStgRH7gl0LF6tw09yuq',
 '5jv2tzybXeTbZJ61jvYvqg',
 '4DZlJSUZnQUFMv4nW6Ijgv']

In [209]:
# Next let's get the names and popularities from these albums
pops = [spotify.album(x).popularity for x in sylvanAlbumList]
names = [spotify.album(x).name for x in sylvanAlbumList]
total = zip(sylvanAlbumList, names, pops)
list(total)

[('6rl4rzZxECegl9PdYG0hfk', 'No Rules Sandy', 49),
 ('2KAPJ40On8JY1Yt64JJgTk', 'Free Love', 49),
 ('68ClkAMsCDyx2lQMsnbY55', 'WITH', 32),
 ('1iKIf39XkjsKa5xCKJmcN4', "Didn't Care", 22),
 ('3RP1mc5yIIwGCrdST276Lg', 'Your Reality', 16),
 ('1203zCqGjsMNe9vre3dMrJ', 'Sunburn', 27),
 ('1Ho3UkeW5DfE812uhE6Q9Q', 'Numb', 28),
 ('1UiIz58AGc20MDp1LMYsdu', 'WITH LOVE', 24),
 ('7m9kZfDONAl0UMYeQRFTNI',
  'Ferris Wheel [Feat. Robert Glasper] (Terrace Martin Remix)',
  33),
 ('0mZc9rzgNi9Jl0PbuU0G3H', 'Ferris Wheel (Maya Jane Coles Remix)', 21),
 ('1n2SGGnNyAy6npmMNxIjuV', 'Free', 9),
 ('4edAJKbnB3xCVsvXmhKvv0', 'Frequency', 21),
 ('3PvStgRH7gl0LF6tw09yuq', 'Rooftop Dancing', 26),
 ('5jv2tzybXeTbZJ61jvYvqg', 'Ferris Wheel', 29),
 ('4DZlJSUZnQUFMv4nW6Ijgv', 'Dark Days [Feat. Sylvan Esso]', 50)]

In [227]:
# Select any of these that are in artist top ten
# Get list of top ten tracks
topTrack = spotify.artist_top_tracks('39vA9YljbnOApXKniLWBZv', market="US")
topTrack[0]

FullTrack with fields:
  album = SimpleAlbum(album_group, album_type, artists, ...)
  artists = [1 x SimpleArtist(external_urls, href, id, name, type, uri)]
  available_markets = None
  disc_number = 1
  duration_ms = 268213
  explicit = False
  external_ids = {'isrc'}
  external_urls = {'spotify'}
  href = 'https://api.spotify.com/v1/tracks/30GGIrrJdSNtecPiFcVP5O'
  id = '30GGIrrJdSNtecPiFcVP5O'
  is_local = False
  is_playable = True
  linked_from = None
  name = 'Coffee'
  popularity = 58
  preview_url = 'https://p.scdn.co/mp3-preview/c432b2b907d3a6dc4917ffc4...'
  restrictions = None
  track_number = 7
  type = 'track'
  uri = 'spotify:track:30GGIrrJdSNtecPiFcVP5O'

In [226]:
# Get release dates for top tracks
[x.album.release_date for x in spotify.artist_top_tracks('39vA9YljbnOApXKniLWBZv', 'US')]

['2014-05-13',
 '2017-04-28',
 '2018-09-07',
 '2014-05-13',
 '2017-04-28',
 '2020-09-25',
 '2014-05-13',
 '2017-04-28',
 '2020-09-25',
 '2017-07-28']

In [228]:
# Make a function that selects any tracks that are in the artist's top ten and which are less than 5 years old
def get_top_5yos(artistID):
    trackList = [
        x.id for x in spotify.artist_top_tracks(artistID, market="US") if datetime.strptime(x.album.release_date, '%Y-%m-%d')>MINDATE
    ]
    return trackList

In [229]:
get_top_5yos('39vA9YljbnOApXKniLWBZv')

['5Y42u37PlyQXegnuI1Hpo1', '0yiJNNm39cp1T7RnXOMVLX', '4q7wwwMXLoIkuOJv3LvbON']

In [23]:
# Read in list of artists
artists = pd.read_csv('artists.csv')
# Subset for testing
artists = artists[:5]
artists

,artistName
0,21 Savage
1,3 Doors Down
2,311
3,A Day To Remember
4,A Tribe Called Quest


In [24]:
# Create function to search spotify based on artist name
def searchSpotify(artists):
    try:
        # Search spotify for artists based on name
        possibleArtists, = spotify.search(artists['artistName'], types=['artist'], limit=1)
        # Select the first search result
        artists['artistID'] = possibleArtists.items[0].id
        artists['artistPop'] = possibleArtists.items[0].popularity
        artists['artistGenre'] = ", ".join(list(possibleArtists.items[0].genres))
        result = artists
    except: 
        artists['artistID'] = np.nan
        artists['artistPop'] = np.nan
        artists['artistGenre'] = np.nan
        result = artists
    finally:
        # Wait 2 seconds so we do not exceed our API call allowance
        time.sleep(2)
        return result

# Find an artistID for each artist based on artistName
artists = artists.apply(searchSpotify, axis=1)
artists

,artistName,songURI,artistID,artistPop,artistGenre
0,21 Savage,test,1URnnhqYAYcrqrcwql10ft,93,"atl hip hop, rap"
1,3 Doors Down,test,2RTUTCvo6onsAnheUk3aL9,72,"alternative metal, nu metal, pop rock, post-gr..."
2,311,test,41Q0HrwWBtuUkJc7C1Rp6K,65,"alternative metal, alternative rock, funk meta..."
3,A Day To Remember,test,4NiJW4q9ichVqL1aUsgGAN,71,"metalcore, pop punk, screamo"
4,A Tribe Called Quest,test,09hVIj6vWgoCDtT03h8ZCa,69,"alternative hip hop, conscious hip hop, east c..."


In [22]:
possibleArtists, = spotify.search('311', types=['artist'], limit=1)
result = possibleArtists.items[0]
id = possibleArtists.items[0].id
popularity = possibleArtists.items[0].popularity
genres = ", ".join(list(possibleArtists.items[0].genres))
result

FullArtist with fields:
  external_urls = {'spotify'}
  followers = Followers(href, total)
  genres = [9 x str]
  href = 'https://api.spotify.com/v1/artists/41Q0HrwWBtuUkJc7C1Rp6K'
  id = '41Q0HrwWBtuUkJc7C1Rp6K'
  images = [3 x Image(height, url, width)]
  name = '311'
  popularity = 65
  type = 'artist'
  uri = 'spotify:artist:41Q0HrwWBtuUkJc7C1Rp6K'

In [12]:
from collections import Counter
testList = ['a','a','a','a','b','b','b','b','b','b','c']
list(dict(sorted(Counter(testList).items(), key=lambda x:x[1], reverse=True)).keys())

['b', 'a', 'c']

In [ ]:
spotify.playlist_items(playlistID)